In [ ]:
#Check directory
#!pwd

In [ ]:
#Change directory
#%cd /Users/georgesmith/Desktop/RTP/Season 28/Development/Analysis/

In [ ]:
#Import necessary packages:
import pandas as pd
from pandas.plotting import scatter_matrix
import numpy as np
import datetime
from datetime import date
from datetime import datetime
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from IPython.display import display
#import itertools
#import sys
#Install pingouin package into this jupyter kernal
#!{sys.executable} -m pip install pingouin
#import pingouin as pg #this throws the warning message
#!{sys.executable} -m pip install --upgrade statsmodels
#import pickle
# import plotly.offline as py
# color = sns.color_palette()
# import plotly.graph_objs as go
# py.init_notebook_mode(connected=True)
# import plotly.tools as tls


%matplotlib inline

In [ ]:
#Read in donation data:
input_cols = ['account_id',
 'account_name',
 'contact_name',
 'account_record_type',
 'do_not_mail',
 'donation_record_type',
 'informal_salutation',
 'amount',
 'close_date',
 'email_opt_out',
 'contact_email',
 'mailing_street',
 'mailing_city',
 'mailing_state',
 'mailing_zip',
 'fund',
 'frequency',
 'payment_type',
 'pledge_amount',
 'payment_schedule',
 'type',
 'stage',
 'fiscal_year',
 'fiscal_quarter',
 'last_donation_amount',
 'last_donation_date',
 'close_month',
 'anonymous',
 'card_type',
 'check_date',
 'donation_matched',
 'grant_amount',
 'matching_amount',
 'matching_close_date',
 'first_donation_date',
 'lifetime_donation_total_amount',
 'lifetime_donation_count',
 'lifetime_subscription_purchase_total',
 'lifetime_single_ticket_purchase_total',
 'lifetime_order_count']
 
#raw_donations = pd.read_csv('/Users/georgesmith/Desktop/RTP/Season 28/Development/Analysis/donations_Jul2016_Jan112021.csv', header=0, names=input_cols)
raw_donations = pd.read_csv('/Users/georgesmith/Desktop/RTP/Season 28/Development/Analysis/donations_Jul2014_Jan202021.csv', header=0, names=input_cols)

In [ ]:
#Expand dataframe display breadth
pd.set_option('display.max_columns', None)
#pd.set_option('display.max_rows', None)

In [ ]:
#Remove columns necessary to anonymize each record (after analysis, come back and remove 'contact_name', 'account_name')
iden_cols = ['account_name', 'contact_name', 'informal_salutation', 'donation_matched', 'contact_email', 'mailing_street']
donations = raw_donations.drop(iden_cols, axis=1, inplace=False)
donations.head()

In [ ]:
donations.shape

In [ ]:
donations.info()

In [ ]:
#Drop column(s) with all missing values ['matching_close_date']
donations.drop('matching_close_date', axis=1, inplace=True)

In [ ]:
def col_dtype_conversions(df, col_list1, new_dtype1, col_list2, new_dtype2):
    """
    Converts columns in col_list1 to datatype specified in new_dtype1
    and converts those in col_list2 to datatype specified in new_dtype2

    Params
    ------
    df : pandas.DataFrame
        dataframe with columns in both col_lists
    col_list1 : list
        list of column names to be converted to dtype specified by new_dtype1
    new_dtype1 : str (must put in quotes)
        data type to which columns in col_list1 will be converted
    col_list2 : list
        list of column names to be converted to dtype specified by new_dtype2
    new_dtype2 : str (must put in quotes)
        data type to which columns in col_list2 will be converted

    Returns
    -------
    pandas.DataFrame
        Returns a dataframe with the same columns, some of which are now cast as
        the specified data types
    """
    for col1 in col_list1:
        df[col1] = df[col1].astype(new_dtype1)
        for col2 in col_list2:
            df[col2] = df[col2].astype(new_dtype2)
    return

In [ ]:
col1 = ['fiscal_year', 'do_not_mail', 'email_opt_out', 'anonymous']
col2 = ['close_date', 'close_month', 'first_donation_date', 'last_donation_date', 'check_date']

In [ ]:
col_dtype_conversions(donations, col1, 'str', col2, 'datetime64[ns]')

In [ ]:
#Select donations where close_date > '2015-07-01'
donations = donations[donations.close_date > '2015-06-30']

In [ ]:
#Parse month from full dates shown in close_month column
donations.close_month = donations.close_month.dt.month.astype('str')

In [ ]:
#Confirm data type changes
donations.info()

In [ ]:
#Review the distribution of the stage column
donations.stage.value_counts(dropna=False)

In [ ]:
#Examine the 'Refunded' records to confirm that they should be dropped from this analysis
donations[donations.stage == 'Refunded']

In [ ]:
#Drop donations marked as 'Refunded' in stage column
donations = donations[donations.stage != 'Refunded'].reset_index(drop=True)

In [ ]:
#Take a closer look at records where stage == any of these: 'Partially Paid', 'Pending', 'Paid in Full', 'Not Posted'
donations[donations.stage.isin(['Partially Paid', 'Pending', 'Paid in Full', 'Not Posted'])].reset_index()

In [ ]:
donations[(donations.stage.isin(['Partially Paid', 'Pending', 'Paid in Full', 'Not Posted'])) &
         (donations.amount.notnull())].reset_index()

In [ ]:
#Since none of the records with an populated amount field and stage field equal to 'Partially Paid', 'Pending', 
#'Paid in Full', or 'Not Posted' are 'Annual Fund' records, drop them
donations = donations[~donations.stage.isin(['Partially Paid', 'Pending', 'Paid in Full', 'Not Posted'])].reset_index(drop=True)

In [ ]:
#display input summary statistics for all columns
donations.describe(include = 'all').T

In [ ]:
#The mean gift of all donations made from 7/1/2015 to 1/10/2021 is
donations.describe().loc['mean', 'amount']

In [ ]:
#The median gift of all donations made from 7/1/2015 to 1/10/2021 is
donations.describe().loc['50%', 'amount']

In [ ]:
#The max gift of all donations made from 7/1/2015 to 1/10/2021 is
donations.describe().loc['max', 'amount']

In [ ]:
donations.fund.value_counts(dropna=False)

In [ ]:
donations[donations.fund.isna()]

In [ ]:
donations[donations.fund.notnull()]

In [ ]:
#Donations with missing values in the fund column are Annual Fund gifts made online -- reassign accordingly
donations.fund.fillna('Annual Fund', inplace=True)

In [ ]:
donations.fund.value_counts(dropna=False)

In [ ]:
#Consolidate all Annual Fund donations under a common string
donations.fund = np.where(donations.fund.isin(['ANN', 'Annual Fund Historical']), 'Annual Fund', donations.fund)

In [ ]:
#Confirm
donations.fund.value_counts(dropna=False)

In [ ]:
#Group levels referencing the same fund under common levels
donations.loc[(donations['fund'].isin(["25th Anniversary Campaign Fund", '25th Anniversary Fund', '25TH'])), 'fund'] = "25th Anniversary Capital Campaign"
donations.loc[(donations['fund'] == 'NEXT'), 'fund'] = "Next Stage Campaign"
donations.loc[(donations['fund'] == 'John Knapp -Tim Gillham Fund'), 'fund'] = "New Works (Knapp-Gillham) Fund"
donations.loc[(donations['fund'].isin(["Education Program Fund", 'EDUC'])), 'fund'] = "Education Fund"
donations.loc[(donations['fund'] == 'MMP'), 'fund'] = "Mary Morton Parsons Challenge Grant Match"

In [ ]:
donations[donations.fund == 'Advertiser']

In [ ]:
#Reviewed cleaned fund column distribution
donations.fund.value_counts(dropna=False)

Finding variation in how Advertising placement 'donations' are stored. For Data Dictionary and updating records/entries going forward: we'll note advertising payments with
#donation_record_type = Ad Deposit / Ad Balance
#fund = Advertiser
#type = Sale

The maximum values in the amount and pledge_amount columns highlight that the dataframe includes donations made by all account_record_types, including foundations and businesses. Because this analysis focuses on individual giving, we'll select only records where account_record_type == ['Individual', ' Household']. 

In [ ]:
donations = donations[donations['account_record_type'].isin(['Individual', 'Household'])].reset_index(drop=True)
donations.head()

In [ ]:
donations.account_record_type.value_counts()

Additionally, with 0 shown as the minimum donation amount in a dataframes where each row is a donation record, we examine those records to determine what information these rows are actually storing. Subsequently, we can decide which, if any, should/can be removed from our donation analysis. UPDATE: E.D. instructs to remove donations where amount == 0 (from PatronManager)

In [ ]:
donations[donations['amount'] == 0]

In [ ]:
#Remove the 2 records where amount == 0 as they don't appear to hold any valuable information.
donations = donations[donations['amount'] != 0]

In [ ]:
#Display summary statistics
donations.describe(include = 'all').T

Because $25,000 seems a bit high for an individual donor's pledge AND grants tend to be awarded by foundations or government, the pledge_amount and grant_amount maximum and count metrics, respectively, stand out among Individual/Household account summary statistics. We'll take a closer look to see what's happening with these. 

In [ ]:
#Examine the donation record that records a $25,000 pledge
donations[donations.pledge_amount == 25000]

The $25,000 pledge is, in fact, accurately entered.

In [ ]:
#Examine the donation record that records a grant
donations[donations.grant_amount.notnull()]

Executive Director tells me this is, in fact, an individual donor's annual fund donation, which is correctly designated as a grant because it comes from their family advised fund.

In [ ]:
#Reassign this record to annual fund donation (i.e. change donation_record_type to Donation & amount to $100)
donations.loc[(donations.account_id == '0013600000ggdlM'), 'last_donation_date'] = '2020-12-21'
donations.loc[(donations.account_id == '0013600000ggdlM'), 'last_donation_amount'] = 100
donations.loc[(donations.account_id == '0013600000ggdlM') & (donations.close_date == '2020-12-21'), 'donation_record_type'] = 'Donation'
donations.loc[(donations.account_id == '0013600000ggdlM') & (donations.close_date == '2020-12-21'), 'amount'] = 100

In [ ]:
donations[donations.account_id == '0013600000ggdlM']

In [ ]:
#Check out the max donations record (from dataset with wider timeframe)
donations[donations.amount == 158115.36]

The 'Next Stage Campaign' donation above reminds us that we'll need to select only annual fund gifts for the analysis once we impute missing values.

In [ ]:
#Check out the max donations record
donations[donations.amount == 25000]

The 'Project Fund' and 'New Works Fund" donations above reminds us that we'll need to select only annual fund gifts for the analysis once we impute missing values.

In [ ]:
#View donation_record_type category counts
donations.donation_record_type.value_counts(dropna=False)

In [ ]:
#Subset the pledge records from the actual donations
pledges = donations[donations.donation_record_type == 'Pledge']

In [ ]:
#Review Corporate Donations to understand why they were made by an/a individual/household account.
donations[donations.donation_record_type == 'Corporate Donation']

These are advertising payments. Filter from analysis and correct in PatronManager.

In [ ]:
#Select only donations applicable to stewardship program (i.e. hh or individual account record type; fund == annual fund; 
#donation records type isin(['Donation', 'PatronTicket Donation', 'Pledge Payment', 'Grant', 'Matching Donation']))
donations = donations[donations.donation_record_type.isin(['Donation', 'PatronTicket Donation', 'Pledge Payment', 'Grant', 'Matching Donation'])]

In [ ]:
donations[donations.amount.isna()]

In [ ]:
#Filter out the donation record(s) id-ed above where amount is nan
donations = donations[donations.amount.notnull()].reset_index(drop=True)

In [ ]:
#Remove PatronManager Client Services records from analysis (i.e account_id == '0013600000TKzRp')
donations = donations[donations.account_id != '0013600000TKzRp']

**How many new donors did we acquire over the period FY 2016-2021 (to 1/20/2021)**

In [ ]:
len(donations[donations['first_donation_date'] >'2015-06-30'].account_id.unique())

In [ ]:
#Show the first time donors
donations.drop_duplicates('account_id')[donations.drop_duplicates('account_id')['first_donation_date'] > '2015-06-30'].reset_index(drop=True)

RTP acquired 1311 new donors during the years under review

In [ ]:
donations.describe(include='all').T

# DOUBLE CHECK THIS STAT

*Noteworthy statistic: 1306 first time donations / acquired donors of the 4314 (30.27%) individual/household account donations we've received over the past 4.5 years have been first-time donations.*

In [ ]:
#Generate summary stats for nonAF donations
donations.loc[donations.fund != 'Annual Fund', :].copy().reset_index(drop=True).describe(include='all').T

*From July 1, 2015 - January 10, 2021, the mean donation across all funds, except for the annual fund, is 3550.5, median 300.*

**Select only the Annual Fund records as they are the primary donations our communications efforts are meant to drive.**

In [ ]:
aFund = donations.loc[donations.fund == 'Annual Fund', :].copy().reset_index(drop=True)

In [ ]:
aFund.payment_type.value_counts(dropna=False)

In [ ]:
#Assign group levels referencing the same payment_type to common levels
aFund.loc[(aFund['payment_type'] == 'CRED'), 'payment_type'] = "Credit Card"
aFund.loc[(aFund['payment_type'] == 'Third Party'), 'payment_type'] = "Credit Card - Third Party"
aFund.loc[(aFund['payment_type'] == 'INKND'), 'payment_type'] = "In Kind"

In [ ]:
aFund.loc[(aFund['payment_type'].isna()) & aFund['check_date'].notnull(), 'payment_type'] = "Check"

In [ ]:
aFund.loc[aFund.payment_type.isna()].sort_values('close_date')

In [ ]:
#Assign the remaining records to check
aFund.loc[aFund['payment_type'].isna(), 'payment_type'] = "Check"

In [ ]:
plt.figure(figsize=[15,6])
sns.countplot(aFund.payment_type)
plt.xticks(rotation=45)

In [ ]:
#Remove the In Kind donations as they are not driven by campaigns that raise money for the annual fund:
aFund = aFund[aFund.payment_type != 'In Kind']

In [ ]:
#Did it work?
aFund[aFund.payment_type == 'In Kind']

Because the donors who gave the in kind donations also donated money, this in kind record removal doesn't impact the acquired donor calculation above.

In [ ]:
aFund.shape

In [ ]:
#Review updated summary statistics
aFund.describe(include = 'all').T

*Only 193 of 4111 (4.69%) individual/household donations were made to funds others than 'Annual Fund'*

In [ ]:
figure, ax = plt.subplots(5,2, figsize=(12,24))

#See the distrubution of the data

sns.distplot(aFund['amount'],ax= ax[0,0])
sns.distplot(aFund['last_donation_amount'],ax=ax[0,1])
sns.distplot(aFund['lifetime_donation_count'],ax= ax[1,0])
sns.distplot(aFund['lifetime_donation_total_amount'],ax= ax[1,1])
sns.distplot(aFund['lifetime_single_ticket_purchase_total'],ax= ax[2,0])
sns.distplot(aFund['lifetime_subscription_purchase_total'],ax=ax[2,1])
sns.countplot(donations['fund'],ax= ax[3,0])
sns.countplot(aFund['close_month'],ax= ax[3,1])
sns.distplot(aFund['lifetime_order_count'],ax= ax[4,0])
sns.countplot(aFund['payment_type'],ax=ax[4,1])


plt.show() #optional line -- eliminates strange text output before plots

In [ ]:
aFund['amount'].hist().plot(kind='bar')
plt.title('Donation Amount Distribution')

In [ ]:
import plotly.graph_objects as go # To Generate Graphs
import plotly.express as px # To Generate box plot for statistical representation
fig = px.box(aFund, x='fiscal_year', y='amount')
fig.show()

In [ ]:
pd.set_option('display.max_rows', None)
aFund.groupby('close_date')['amount'].sum().reset_index().sort_values('close_date')

In [ ]:
#Plot the daily annual fund donation totals

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.lineplot(x=aFund.groupby('close_date')['amount'].sum().index, y=aFund.groupby('close_date')['amount'].sum().values, color='green')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Close Date', fontsize=14)
plt.title('Annual Fund Donation Totals Per Day', fontsize=16)

In [ ]:
#Visualize donations over time
dons = aFund.plot.scatter('close_date', 'amount')
plt.title("Donation Amounts By Close Date (FY2016 - FY2021)")

In [ ]:
aFund[aFund.amount < 6000].plot('close_date', 'amount', kind='scatter', alpha=.35)

In [ ]:
aFund[aFund.amount < 1001].plot('close_date', 'amount', kind='scatter', alpha=.25)

In [ ]:
aFund.groupby('close_date')['amount'].sum()

In [ ]:
#Plot the daily annual fund donation totals

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.swarmplot(x=aFund.groupby('close_date')['amount'].sum().index, y=aFund.groupby('close_date')['amount'].sum().values, color='magenta')

#Assign axis labels:
plt.ylabel('Donation Total ($)', fontsize=14)
plt.xlabel('Close Date', fontsize=14)
plt.title('Annual Fund Donation Totals Per Day')

In [ ]:
aFund.groupby('close_date')['amount'].sum().sort_values()

In [ ]:
ax = aFund.groupby('close_date')['amount'].sum().hist(bins = 40).plot(kind='bar')
plt.title('Donation Amount Distribution')

In [ ]:
#The mean gift of all donations made from 7/1/2015 to 1/10/2021 is
aFund.describe().loc['mean', 'amount']

In [ ]:
#The median gift of all donations made from 7/1/2015 to 1/10/2021 is
aFund.describe().loc['50%', 'amount']

In [ ]:
#The mean gift of all donations made from 7/1/2015 to 1/10/2021 is
aFund.describe().loc['max', 'amount']

# Examine Individual Donor Engagement Patterns

Determine which and how many individual/household donor accounts made annual fund contributions in each of the fiscal year (2016-2021). Additionally, show how many donations each account made, along with their sums, per fiscal year.

In [ ]:
#How many annual fund contributions did each donor make over the period?
aFund.groupby(['account_id'])['fiscal_year'].count().sort_values(ascending=False)

In [ ]:
aFund.groupby(['account_id'])['fiscal_year'].count().sort_values(ascending=False).values

In [ ]:
aFund.groupby(['fiscal_year', 'account_id']).count().sort_values(by=['fiscal_year', 'account_id'])

In [ ]:
#How many annual fund donations were made in each fiscal year over the analysis period
aFund.groupby(['fiscal_year'])['account_id'].count()

In [ ]:
aFund.groupby(['fiscal_year'])['amount'].count()

In [ ]:
aFund[aFund['fiscal_year'] == '2016'].reset_index(drop=True)

In [ ]:
#How many annual fund donors by fy?
aFund.groupby(['fiscal_year'])['account_id'].nunique()

In [ ]:
aFund[aFund['fiscal_year'] == '2016'].drop_duplicates('account_id').reset_index(drop=True)

In [ ]:
#What's the average number of annual fund gifts given in each fiscal year from FY2016- FY2021 (current date of)
aFund.groupby(['fiscal_year'])['account_id'].count().values.mean()

In [ ]:
#average before last fiscal year
aFund.groupby(['fiscal_year'])['account_id'].count().values[:4].mean()

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[10,5])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_year'])['account_id'].count().index, y=aFund.groupby(['fiscal_year'])['account_id'].count().values, 
            hue=aFund.groupby(['fiscal_year'])['amount'].count().loc[['2016', '2017', '2018', '2019', '2020', '2021']].index,
            dodge=False)
g.legend().set_title('RTP Fiscal Year')

#Assign axis labels:
plt.ylabel('Number of Donations', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Counts By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 0
# Annotating the bar plot with the values (means by fy)
for i in range(len(['2016', '2017', '2018', '2019', '2020', '2021'])):
    plt.annotate(round(aFund.groupby(['fiscal_year'])['account_id'].count().loc[['2016', '2017', '2018', '2019', '2020', '2021']], 2).values[i],
                 (-0.1 + i, aFund.groupby(['fiscal_year'])['account_id'].count().loc[['2016', '2017', '2018', '2019', '2020', '2021']].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[10,5])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_year'])['account_id'].nunique().index, y=aFund.groupby(['fiscal_year'])['account_id'].nunique().values, 
            hue=aFund.groupby(['fiscal_year'])['account_id'].nunique().loc[['2016', '2017', '2018', '2019', '2020', '2021']].index,
            dodge=False)
g.legend().set_title('RTP Fiscal Year')

#Assign axis labels:
plt.ylabel('Number of Unique Donors (Account IDs)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donor Volume By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 0
# Annotating the bar plot with the values (means by fy)
for i in range(len(['2016', '2017', '2018', '2019', '2020', '2021'])):
    plt.annotate(round(aFund.groupby(['fiscal_year'])['account_id'].nunique().loc[['2016', '2017', '2018', '2019', '2020', '2021']], 2).values[i],
                 (-0.1 + i, aFund.groupby(['fiscal_year'])['account_id'].nunique().loc[['2016', '2017', '2018', '2019', '2020', '2021']].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_year'])['amount'].sum().index, y=aFund.groupby(['fiscal_year'])['amount'].sum().values, 
            hue=aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']].index,
            dodge=False)
g.legend().set_title("RTP_Fiscal_Year")

#Assign axis labels:
plt.ylabel('Donation Totals ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 10
# Annotating the bar plot with the values (means by fy)
for i in range(len(np.sort(aFund.fiscal_year.unique()))):
    plt.annotate(round(aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']], 2).values[i],
                 (-0.1 + i, aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
sns.boxplot(x='fiscal_year',y='amount',data=aFund[aFund.amount <2000])

In [ ]:
sns.boxplot(x='fiscal_year',y='amount',data=aFund[aFund.amount <500])

In [ ]:
aFund.groupby(['fiscal_year'])['amount'].median()

In [ ]:
#Plot the annual fund medians by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=aFund.groupby(['fiscal_year'])['amount'].median().index, y=aFund.groupby(['fiscal_year'])['amount'].median().values, color='green')

#Assign axis labels:
plt.ylabel('Median Donations ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Median By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 0
# Annotating the bar plot with the values (means by fy)
for i in range(len(np.sort(aFund.fiscal_year.unique()))):
    plt.annotate(round(aFund.groupby(['fiscal_year'])['amount'].median().loc[np.sort(aFund.fiscal_year.unique())], 2).values[i],
                 (-0.1 + i, aFund.groupby(['fiscal_year'])['amount'].median().loc[np.sort(aFund.fiscal_year.unique())].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
#Plot the annual fund medians by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=aFund.groupby(['fiscal_year'])['amount'].mean().index, y=aFund.groupby(['fiscal_year'])['amount'].mean().values, color='green')

#Assign axis labels:
plt.ylabel('Mean Donations ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Mean By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 0
# Annotating the bar plot with the values (means by fy)
for i in range(len(np.sort(aFund.fiscal_year.unique()))):
    plt.annotate(round(aFund.groupby(['fiscal_year'])['amount'].mean().loc[np.sort(aFund.fiscal_year.unique())], 2).values[i],
                 (-0.1 + i, aFund.groupby(['fiscal_year'])['amount'].mean().loc[np.sort(aFund.fiscal_year.unique())].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
fq_wfy = pd.DataFrame(aFund.groupby(['fiscal_quarter'])['amount'].mean().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']])
#Add 'FY' to year to clarify the legend labels
fq_wfy['fiscal_year'] = np.concatenate([np.repeat('FY 2016', 4), np.repeat('FY 2017', 4), np.repeat('FY 2018', 4), np.repeat('FY 2019', 4), np.repeat('FY 2020', 4), np.repeat('FY 2021', 3)])
fq_wfy['fiscal_year']#Create fiscal year array for coloring bars based on fiscal year

In [ ]:
fq_wfy

In [ ]:
aFund.groupby(['fiscal_quarter'])['amount'].sum()

In [ ]:
#Plot the annual fund totals by fiscal quarter

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_quarter'])['amount'].count().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].index, 
            y=aFund.groupby(['fiscal_quarter'])['amount'].count().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values, 
            hue = fq_wfy['fiscal_year'].values,
           dodge=False)
g.legend(loc='upper left').set_title('RTP Fiscal Year')
#Assign axis labels:
plt.ylabel('Number of Donations', fontsize=14)
plt.xlabel('Fiscal Quarter', fontsize=14)
plt.xticks(rotation=45)
plt.title('Annual Fund Donation Counts By Fiscal Quarter', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 6
# Annotating the bar plot with the values 
for i in range(len(['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021'])):
    plt.annotate(aFund.groupby(['fiscal_quarter'])['amount'].count().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i], (-0.1 + i, aFund.groupby(['fiscal_quarter'])['amount'].count().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i] + j),
                horizontalalignment='center',
                 fontsize=13)

In [ ]:
#Plot the annual fund totals by fiscal quarter

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[20,10])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].index, 
            y=aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values, 
            hue = fq_wfy['fiscal_year'].values,
           dodge=False)
g.legend(loc='upper left').set_title('RTP Fiscal Year')
#Assign axis labels:
plt.ylabel('Donations Totals ($)', fontsize=16)
plt.xlabel('Fiscal Quarter', fontsize=16)
plt.xticks(rotation=45, fontsize=15)
plt.yticks(fontsize=15)
plt.title('Annual Fund Donation Totals By Fiscal Quarter', fontsize=17)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 2000
# Annotating the bar plot with the values 
for i in range(len(['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021'])):
    plt.annotate(round(aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']], 3).values[i], (-0.1 + i, aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i] + j), 
                 horizontalalignment='center',
                 verticalalignment='top',
                 fontsize=13)

In [ ]:
#Plot the annual fund medians by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_quarter'])['amount'].median().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].index, 
            y=aFund.groupby(['fiscal_quarter'])['amount'].median().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values, 
            hue = fq_wfy['fiscal_year'].values,
           dodge=False)
#Add legend title (line below + setting the sns.barplot object equal to 'g' as above)
g.legend().set_title('RTP Fiscal Year')

#Assign axis labels:
plt.ylabel('Median Donation ($)', fontsize=14)
plt.xlabel('Fiscal Quarter', fontsize=14)
plt.xticks(rotation=45)
plt.title('Annual Fund Donation Median By Fiscal Quarter', fontsize=15)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 1
# Annotating the bar plot with the values 
for i in range(len(['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021'])):
    plt.annotate(round(aFund.groupby(['fiscal_quarter'])['amount'].median().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']], 2).values[i], (-0.1 + i, aFund.groupby(['fiscal_quarter'])['amount'].median().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i] + j),
                horizontalalignment='center',
                fontsize=13)

In [ ]:
fq_mean = (aFund.groupby(['fiscal_quarter'])['amount'].mean().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']])

In [ ]:
#Plot the annual fund means by fiscal quarter

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot

g = sns.barplot(x=fq_mean.index, 
            y=fq_mean.values,
           hue = fq_wfy['fiscal_year'].values,
           dodge=False)
g.legend(loc='upper right').set_title('RTP Fiscal Year')
#Assign axis labels:
plt.ylabel('Mean Donation ($)', fontsize=14)
plt.xlabel('Fiscal Quarter', fontsize=14)
plt.xticks(rotation=45)
plt.title('Annual Fund Donation Mean By Fiscal Quarter', fontsize=15)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 4
# Annotating the bar plot with the values 
for i in range(len(['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021'])):
    plt.annotate(round(aFund.groupby(['fiscal_quarter'])['amount'].mean().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']], 2).values[i], (-0.1 + i, aFund.groupby(['fiscal_quarter'])['amount'].mean().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i] + j),
                horizontalalignment='center',
                fontsize=12.5)

**INTERESTING THAT EARLIER QUARTERS HAVE HIGHER MEAN AND MEDIAN DONATION AMOUNTS GIVEN THEY ALSO TEND TO HAVE FEWER DONATIONS AND LOWER TOTALS. THIS INSIGHT MAY BE DUE TO OUR BEGINNING TO TAKE PART IN DEVELOPMENT INITIATIVES THAT AWARD PRIZES BASED ON NUMBER OF DONORS -- THESE EFFORTS EXPLAIN AT LEAST A GOOD PORTION OF THE REASON LATER QUARTERS HAVE LOWER AVG METRICS**

In [ ]:
#Combine aggregations of interest into a single df
#Create dictionary of aggregations needed to summarize data as desired.
aggregations = {
    'number_of_donations_byFY' : ('account_id', 'size'),
    'donor_total_byFY' : ('amount', 'sum'),
    'donor_median_byFY' : ('amount', 'median'),
    'donor_avg_byFY' : ('amount', 'mean'),
    'donor_max_byFY' : ('amount', 'max'),
    'donor_min_byFY' : ('amount', 'min'),
    'donor_last_gift_date' : ('last_donation_date', 'max'),
    'donor_first_gift_date' : ('first_donation_date', 'min')
    #'don_recency_median' : ('donation_recency', 'median'),
    #'don_recency_avg' : ('donation_recency', 'mean'),
    #'last_don_median' : ('last_donation_amount', 'median'),
    #'last_don_avg' : ('last_donation_amount', 'median'),
    #'avg_days_since_last' : ('days_since_last', 'mean'),
    #'median_days_since_last' : ('days_since_last', 'median'),
    #'avg_days_since_first' : ('days_since_first', 'mean'),
    #'median_days_since_first' : ('days_since_first', 'median'),
    #'number_firsts_byFY' : ('first_donation_flag', 'sum'),
    #'number_lastsbyFY' : ('last_donation_flag', 'sum')
}

In [ ]:
donors_by_fy = aFund.groupby(['fiscal_year', 'account_id']).agg(**aggregations).reset_index()
donors_by_fy.head(25)

In [ ]:
donors_by_fy.isna().sum()

In [ ]:
#Which donors gave in each of the fiscal years under review?
donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017',:].sort_values('donor_last_gift_date')

In [ ]:
len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values)

In [ ]:
#Double check the number of unique accounts across the entire period
len(aFund.account_id.unique())

# Create data table of retention metrics

**Create arrays of the each fiscal year's retained account ids.**

In [ ]:
retained_2021 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2021', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values])
retained_2020 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values])
retained_2019 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values])
retained_2018 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values])
retained_2017 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values])

**Calculate Retention Rates by Fiscal Year**

In [ ]:
# retained_2021 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2021', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values])
# retained_2020 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values])
# retained_2019 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values])
# retained_2018 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values])
# retained_2017 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values if a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values])

In [ ]:
len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values)

In [ ]:
len(retained_2021)

In [ ]:
#2021 retention rate
round(len(retained_2021)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values)*100, 2)

**Create arrays of each fiscal year's acquired account ids.**

In [ ]:
donors_by_fy[(donors_by_fy['donor_first_gift_date'] > '2020-06-30') & (donors_by_fy['fiscal_year'] == '2021')].sort_values('donor_first_gift_date').reset_index(drop=True)

In [ ]:
acquired_2021 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] > '2020-06-30') & (donors_by_fy['fiscal_year'] == '2021'), 'account_id'].values
len(acquired_2021)

In [ ]:
acquired_2020 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] > '2019-06-30') & (donors_by_fy['donor_first_gift_date'] < '2020-07-01') & (donors_by_fy['fiscal_year'] == '2020'), 'account_id'].values
len(acquired_2020)

In [ ]:
acquired_2019 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] > '2018-06-30') & (donors_by_fy['donor_first_gift_date'] < '2019-07-01') & (donors_by_fy['fiscal_year'] == '2019'), 'account_id'].values
len(acquired_2019)

In [ ]:
donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] > '2017-06-30') & (donors_by_fy['donor_first_gift_date'] < '2018-07-01') & (donors_by_fy['fiscal_year'] == '2018')].sort_values('donor_first_gift_date').reset_index(drop=True)

In [ ]:
acquired_2018 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] > '2017-06-30') & (donors_by_fy['donor_first_gift_date'] < '2018-07-01') & (donors_by_fy['fiscal_year'] == '2018'), 'account_id'].values
len(acquired_2018)

In [ ]:
acquired_2017 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] > '2016-06-30') & (donors_by_fy['donor_first_gift_date'] < '2017-07-01') & (donors_by_fy['fiscal_year'] == '2017'), 'account_id'].values
len(acquired_2017)

**Create arrays of the each fiscal year's reactivated account ids.**

In [ ]:
donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] < '2016-06-30') & (donors_by_fy['donor_last_gift_date'] > '2020-06-30')].reset_index(drop=True)

In [ ]:
#Reactivated in FY2017:
reactivated_2017 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] <= '2015-06-30') & (donors_by_fy['fiscal_year'] == '2017') & (~donors_by_fy['account_id'].isin(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values))]['account_id'].values

In [ ]:
#Reactivated in FY2018:
reactivated_2018 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] <= '2016-06-30') & (donors_by_fy['fiscal_year'] == '2018') & (~donors_by_fy['account_id'].isin(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values))].sort_values('donor_first_gift_date')['account_id'].values

In [ ]:
#Reactivated in FY2019:
reactivated_2019 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] <= '2017-06-30') & (donors_by_fy['fiscal_year'] == '2019') & (~donors_by_fy['account_id'].isin(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values))].sort_values('donor_first_gift_date')['account_id'].values

In [ ]:
#Reactivated in FY2020:
reactivated_2020 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] <= '2018-06-30') & (donors_by_fy['fiscal_year'] == '2020') & (~donors_by_fy['account_id'].isin(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values))].sort_values('donor_first_gift_date')['account_id'].values

In [ ]:
#Reactivated in FY2021:
reactivated_2021 = donors_by_fy.loc[(donors_by_fy['donor_first_gift_date'] <= '2019-06-30') & (donors_by_fy['fiscal_year'] == '2021') & (~donors_by_fy['account_id'].isin(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values))].sort_values('donor_first_gift_date')['account_id'].values

**Create arrays of the each fiscal year's lapsed account ids.**

In [ ]:
lapsed_2021 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values if a not in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2021', 'account_id'].values])
lapsed_2020 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values if a not in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values])
lapsed_2019 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values if a not in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values])
lapsed_2018 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values if a not in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values])
lapsed_2017 = ([a for a in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values if a not in donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values])

In [ ]:
#Confirm that np.tile will, in fact, repeat the array elements 5 times while retaining the impute order with each iteration
#Unlike np.repeat, which seems to sort like array values together
np.tile(aFund.account_id.unique(), 5)

In [ ]:
round(len(lapsed_2017)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values)*100, 2)

In [ ]:
round(len(retained_2017)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values)*100, 2)

In [ ]:
#Create a new dataframe to store the retention metrics
donor_accounts =pd.DataFrame()
donor_accounts['account_id'] = np.tile(aFund.account_id.unique(), 5) #create 1 row for every unique account for each fiscal year
donor_accounts['fiscal_year'] = np.concatenate([np.repeat('2017', len(aFund.account_id.unique())), np.repeat('2018', len(aFund.account_id.unique())), np.repeat('2019', len(aFund.account_id.unique())), np.repeat('2020', len(aFund.account_id.unique())), np.repeat('2021', len(aFund.account_id.unique()))])
#Add retention rates by fy as column in donor_accounts:
donor_accounts['ret_rate_xfy'] = np.concatenate([np.repeat(round(len(retained_2017)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),                              
                                                 np.repeat(round(len(retained_2018)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),
                                                 np.repeat(round(len(retained_2019)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values)*100, 2), len(aFund.account_id.unique())), 
                                                 np.repeat(round(len(retained_2020)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),
                                                 np.repeat(round(len(retained_2021)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values)*100, 2), len(aFund.account_id.unique()))], axis=0)

donor_accounts['lapse_rate_xfy'] = np.concatenate([np.repeat(round(len(lapsed_2017)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2016', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),
np.repeat(round(len(lapsed_2018)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2017', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),
np.repeat(round(len(lapsed_2019)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2018', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),
np.repeat(round(len(lapsed_2020)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2019', 'account_id'].values)*100, 2), len(aFund.account_id.unique())),
np.repeat(round(len(lapsed_2021)/len(donors_by_fy.loc[donors_by_fy['fiscal_year'] == '2020', 'account_id'].values)*100, 2), len(aFund.account_id.unique()))], axis=0)
donor_accounts.tail()

In [ ]:
donor_accounts.head()

In [ ]:
donor_accounts['ret_rate_xfy'].value_counts()

In [ ]:
donor_accounts['lapse_rate_xfy'].value_counts()

In [ ]:
donor_accounts.shape

In [ ]:
#Add the ret_rate_xfy and lapse_rate_xfy columns to donors_by_fy df
donor_accounts = pd.merge(donor_accounts, donors_by_fy, how='left', on=['account_id', 'fiscal_year']).sort_values(by=['fiscal_year', 'account_id']).reset_index(drop=True)
donor_accounts.tail(20)

In [ ]:
donor_accounts[donor_accounts['number_of_donations_byFY'].isna()].reset_index(drop=True)

In [ ]:
donor_accounts[donor_accounts['number_of_donations_byFY'].notnull()].reset_index(drop=True)

In [ ]:
#How many annual fund donations were given over the period.
donor_accounts['number_of_donations_byFY'].sum()

In [ ]:
#Column addition doesn't show up when checking the donor_accounts.columns. Add it a different way and check
donor_accounts['retention_status'] = 'awaiting value'
donor_accounts.loc[donor_accounts['number_of_donations_byFY'].isna(), 'retention_status'] = np.nan

In [ ]:
#Assign values to new retention_status column based on conditions.
conditions = [
    ((donor_accounts['account_id'].isin(retained_2021)) & (donor_accounts.fiscal_year == '2021')),
    ((donor_accounts['account_id'].isin(lapsed_2021)) & (donor_accounts.fiscal_year == '2021')),
    ((donor_accounts['account_id'].isin(acquired_2021)) & (donor_accounts.fiscal_year == '2021')),
    ((donor_accounts['account_id'].isin(retained_2020)) & (donor_accounts.fiscal_year == '2020')),
    ((donor_accounts['account_id'].isin(lapsed_2020)) & (donor_accounts.fiscal_year == '2020')),
    ((donor_accounts['account_id'].isin(acquired_2020)) & (donor_accounts.fiscal_year == '2020')),
    ((donor_accounts['account_id'].isin(retained_2019)) & (donor_accounts.fiscal_year == '2019')),
    ((donor_accounts['account_id'].isin(lapsed_2019)) & (donor_accounts.fiscal_year == '2019')),
    ((donor_accounts['account_id'].isin(acquired_2019)) & (donor_accounts.fiscal_year == '2019')),
    ((donor_accounts['account_id'].isin(retained_2018)) & (donor_accounts.fiscal_year == '2018')),
    ((donor_accounts['account_id'].isin(lapsed_2018)) & (donor_accounts.fiscal_year == '2018')),
    ((donor_accounts['account_id'].isin(acquired_2018)) & (donor_accounts.fiscal_year == '2018')),
    ((donor_accounts['account_id'].isin(retained_2017)) & (donor_accounts.fiscal_year == '2017')),
    ((donor_accounts['account_id'].isin(lapsed_2017)) & (donor_accounts.fiscal_year == '2017')),
    ((donor_accounts['account_id'].isin(acquired_2017)) & (donor_accounts.fiscal_year == '2017'))
]

choices = ['Retained', 'Lapsed', 'Acquired', 'Retained', 'Lapsed', 'Acquired', 'Retained', 'Lapsed', 'Acquired', 'Retained', 'Lapsed', 'Acquired', 'Retained', 'Lapsed', 'Acquired']

donor_accounts['retention_status'] = np.select(conditions, choices, default=donor_accounts['retention_status'])

**Assign 'Reactivated' status to appropriate rows in the new column**

In [ ]:
donor_accounts.loc[donor_accounts.account_id.isin(reactivated_2017) & (donor_accounts.fiscal_year == '2017'), 'retention_status'] = 'Reactivated'

In [ ]:
donor_accounts.loc[(donor_accounts['account_id'].isin(reactivated_2018)) & (donor_accounts['fiscal_year'] == '2018'), 'retention_status'] = 'Reactivated'

In [ ]:
donor_accounts.loc[(donor_accounts['account_id'].isin(reactivated_2019)) & (donor_accounts['fiscal_year'] == '2019'), 'retention_status'] = 'Reactivated'

In [ ]:
donor_accounts.loc[(donor_accounts['account_id'].isin(reactivated_2020)) & (donor_accounts['fiscal_year'] == '2020'), 'retention_status'] = 'Reactivated'

In [ ]:
donor_accounts.loc[(donor_accounts['account_id'].isin(reactivated_2021)) & (donor_accounts['fiscal_year'] == '2021'), 'retention_status'] = 'Reactivated'

In [ ]:
donor_accounts['retention_status'].value_counts(dropna=False)

# Visualize Retention Status (including, by fiscal year)

In [ ]:
#Plot retention status values counts
donor_accounts['retention_status'].value_counts().plot(kind='bar')

In [ ]:
#Plot retention status by fiscal year
donor_accounts.pivot_table(index='fiscal_year', columns='retention_status', aggfunc='size').plot.bar()

In [ ]:
donor_accounts.pivot_table(index='fiscal_year', columns='retention_status', aggfunc='size').plot.barh()

In [ ]:
donor_accounts.pivot_table(index='retention_status', columns='fiscal_year', aggfunc='size').plot.bar()
plt.show()

In [ ]:
g = donor_accounts.pivot_table(index='retention_status', columns='fiscal_year', aggfunc='size').plot.bar(stacked='True')
g.legend(loc='upper center').set_title("RTP Fiscal Year")
plt.show()

In [ ]:
g = donor_accounts.loc[donor_accounts['retention_status'] != 'Lapsed'].pivot_table(index='fiscal_year', columns='retention_status', aggfunc='size').plot.bar(stacked='True')
plt.title("RTP Enhances Donor Engagement Efficacy Over Time")
plt.xlabel('Fiscal Year (July 1 - June 30)')
plt.ylabel('Current Donor Counts By Retention Status')
plt.xticks(rotation=45)
g.legend().set_title("Retension Status")
plt.show()

In [ ]:
donor_accounts.loc[donor_accounts['retention_status'] != 'Lapsed'].pivot_table(index='fiscal_year', columns='retention_status', aggfunc='size').plot.bar()
plt.show()

In [ ]:
donor_accounts.describe(include = 'all').T

In [ ]:
donor_accounts.isna().sum()

In [ ]:
pd.crosstab(donor_accounts['fiscal_year'], donor_accounts['retention_status'], margins=True)

In [ ]:
g = donor_accounts.loc[donor_accounts['retention_status'] != 'Lapsed'].pivot_table(values = 'donor_total_byFY', index='fiscal_year', columns='retention_status', aggfunc='sum').plot(kind='bar', stacked='True')
plt.title("Fundraising Tactics Grow Donation Totals Year over Year")
plt.xlabel('Fiscal Year (July 1 - June 30)')
plt.ylabel('Current Donor Totals By Retention Status')
plt.xticks(rotation=45)
g.legend().set_title("Retention_Status")
plt.show()

In [ ]:
donor_accounts.loc[donor_accounts['retention_status'] != 'Lapsed'].pivot_table(values = 'donor_total_byFY', index='fiscal_year', columns='retention_status', aggfunc='sum').plot(kind='bar')

# Return here to fix error below and effectively label the bars in the plot. See page #270 in Python for DS

In [ ]:
fig, ax = plt.subplots(figsize=(16,20))
donor_accounts.pivot_table(values = 'donor_total_byFY', index='fiscal_year', columns='retention_status', aggfunc='size', fill_value=0).plot(ax=ax, kind='bar', stacked=True, title='Annual Fund Donor Account Retention Status by Fiscal Year')

style = dict(size=10, color='blue')
#ax.text('Q1-2017', 31, '31', **style)

# Stewardship Program##

Create necessary columns to assign donor levels.

In [ ]:
#Only Annual Fund dontations made by Household or Individual donors are included in stewardship program totals

aFund['stp_total_s28'] = aFund.loc[(aFund['close_date'] >= '2019-07-01')].groupby('account_id')['amount'].transform('sum')
aFund['stp_total_s27'] = aFund.loc[(aFund['close_date'] >= '2018-07-01') & (aFund['close_date'] <= '2020-06-30')].groupby('account_id')['amount'].transform('sum')
aFund['stp_total_s26'] = aFund.loc[(aFund['close_date'] >= '2017-07-01') & (aFund['close_date'] <= '2019-06-30')].groupby('account_id')['amount'].transform('sum')
aFund['stp_total_s25'] = aFund.loc[(aFund['close_date'] >= '2016-07-01') & (aFund['close_date'] <= '2018-06-30')].groupby('account_id')['amount'].transform('sum')
aFund['stp_total_s24'] = aFund.loc[(aFund['close_date'] >= '2015-07-01') & (aFund['close_date'] <= '2017-06-30')].groupby('account_id')['amount'].transform('sum')
#The data does not include the complete date ranges below that are necessary to create the variables commented below
#aFund['stp_total_s23'] = aFund.loc[(aFund['close_date'] >= '2014-07-01') & (aFund['close_date'] <= '2016-06-30')].groupby('account_id')['amount'].transform('sum')

In [ ]:
# aFund['stp_total_s24'] = aFund.loc[(aFund['close_date'] >= '2015-07-01') & (aFund['close_date'] <= '2017-06-30')].groupby('account_id')['amount'].transform('sum')
# aFund['stp_toDate_s24'] = aFund.loc[(aFund['close_date'] >= '2015-07-01') & (aFund['close_date'] <= '2017-01-20')].groupby('account_id')['amount'].transform('sum')

In [ ]:
#Create a another similar set of columns to generate comparable (date range thru 1/11/YY) period over period comparisons

aFund['stp_toDate_s28'] = aFund.loc[(aFund['close_date'] >= '2019-07-01')].groupby('account_id')['amount'].transform('sum')
aFund['stp_toDate_s27'] = aFund.loc[(aFund['close_date'] >= '2018-07-01') & (aFund['close_date'] <= '2020-01-20')].groupby('account_id')['amount'].transform('sum')
aFund['stp_toDate_s26'] = aFund.loc[(aFund['close_date'] >= '2017-07-01') & (aFund['close_date'] <= '2019-01-20')].groupby('account_id')['amount'].transform('sum')
aFund['stp_toDate_s25'] = aFund.loc[(aFund['close_date'] >= '2016-07-01') & (aFund['close_date'] <= '2018-01-20')].groupby('account_id')['amount'].transform('sum')
aFund['stp_toDate_s24'] = aFund.loc[(aFund['close_date'] >= '2015-07-01') & (aFund['close_date'] <= '2017-01-20')].groupby('account_id')['amount'].transform('sum')

In [ ]:
#Fill NaNs with max value for that account_id shown in other rows for that particular stp_total_sXX column 
stp_total_cols = ['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_toDate_s28', 'stp_toDate_s27', 'stp_toDate_s26', 'stp_toDate_s25', 'stp_toDate_s24']
for col in stp_total_cols:
    aFund.loc[aFund[col].isnull(), col] = aFund.groupby('account_id')[col].transform('max')

In [ ]:
#Now, fill remaining nan in stp_total_sXX cols with 0 to simplify ufunc (stp_level_generator) below:  
aFund.loc[:,['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_toDate_s28', 'stp_toDate_s27', 'stp_toDate_s26', 'stp_toDate_s25', 'stp_toDate_s24']] = aFund[['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 'stp_toDate_s28', 'stp_toDate_s27', 'stp_toDate_s26', 'stp_toDate_s25', 'stp_toDate_s24']].fillna(0)

In [ ]:
aFund.isna().sum()

In [ ]:
#Review a selection of account_id that made more than one donation to ensure previous two steps executed as expected
#pd.set_option('display.max_rows', None)
#aFund.loc[aFund.drop_duplicates(subset=['account_id'], keep='last', inplace=True),:].sort_values(by='account_id').reset_index(drop=True)[50:100]

In [ ]:
aFund.describe(include='all').T

In [ ]:
aFund.shape

In [ ]:
aFund.drop(['pledge_amount', 'payment_schedule', 'grant_amount'], axis=1, inplace=True)

In [ ]:
#Define function to translate stewardship sums by season into stewardship levels by account_id
def stp_level_generator(series):
    if series == 0:
        return 'Did Not Give'
    elif (series > 0) & (series < 125):
        return 'Pre-Steward'
    elif (series >= 125) & (series < 250):
        return 'Supporter'
    elif (series >= 250) & (series < 500):
        return 'Investor'
    elif (series >= 500) & (series < 1000):
        return 'Underwriter'
    elif (series >= 1000) & (series < 2500):
        return 'Performer'
    elif (series >= 2500) & (series < 5000):
        return 'Director'
    elif (series >= 5000):
        return 'Producer'

In [ ]:
#Create new dataframe with account_id, stp_total_sX cols and corresponding stp_level cols: 
stp_amt_list = ['stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25', 'stp_total_s24', 
                'stp_toDate_s28', 'stp_toDate_s27', 'stp_toDate_s26', 'stp_toDate_s25', 'stp_toDate_s24']
levels_by_season_df = pd.DataFrame()
levels_by_season_df['account_id'] = aFund['account_id'].copy()

for col in stp_amt_list:
    levels_by_season_df[col] = aFund[col]
    col_name = col + '_level'
    levels_by_season_df[col_name] = (aFund[col].apply(stp_level_generator))

In [ ]:
levels_by_season_df.head()

In [ ]:
levels_by_season_df.shape

In [ ]:
#Remove duplicate rows based on account_id to show each donor's stewardship levels across the most recent 5 seasons 
#(both for the total season and toDate)
levels_by_season_df = levels_by_season_df.drop_duplicates(subset='account_id', keep='first').reset_index(drop='True')
levels_by_season_df.shape

**Add the stewardship totals and levels to the donor_accounts dataframe based on fiscal year**

In [ ]:
levels_fy2021 = levels_by_season_df[['account_id', 'stp_total_s28', 'stp_total_s28_level']].copy()
levels_fy2021.columns = ['account_id', 'stp_total', 'stp_level']
levels_fy2021.insert(1, 'fiscal_year', np.repeat('2021', len(aFund.account_id.unique())))
levels_fy2021.shape

In [ ]:
levels_fy2020 = levels_by_season_df[['account_id', 'stp_total_s27', 'stp_total_s27_level']].copy()
levels_fy2020.columns = ['account_id', 'stp_total', 'stp_level']
levels_fy2020.insert(1, 'fiscal_year', np.repeat('2020', len(aFund.account_id.unique())))
levels_fy2020.shape

In [ ]:
levels_fy2019 = levels_by_season_df[['account_id', 'stp_total_s26', 'stp_total_s26_level']].copy()
levels_fy2019.columns = ['account_id', 'stp_total', 'stp_level']
levels_fy2019.insert(1, 'fiscal_year', np.repeat('2019', len(aFund.account_id.unique())))
levels_fy2019.shape

In [ ]:
levels_fy2018 = levels_by_season_df[['account_id', 'stp_total_s25', 'stp_total_s25_level']].copy()
levels_fy2018.columns = ['account_id', 'stp_total', 'stp_level']
levels_fy2018.insert(1, 'fiscal_year', np.repeat('2018', len(aFund.account_id.unique())))
levels_fy2018.shape

In [ ]:
levels_fy2017 = levels_by_season_df[['account_id', 'stp_total_s24', 'stp_total_s24_level']].copy()
levels_fy2017.columns = ['account_id', 'stp_total', 'stp_level']
levels_fy2017.insert(1, 'fiscal_year', np.repeat('2017', len(aFund.account_id.unique())))
levels_fy2017.shape

In [ ]:
#Append the rows to combine the 5 dfs into one -- merge this with donor_accounts df to 
levels_fy2017.append([levels_fy2018, levels_fy2019, levels_fy2020, levels_fy2021]).reset_index(drop=True)

In [ ]:
#Combine levels df with donor_accounts df
donor_details = pd.merge(donor_accounts, levels_fy2017.append([levels_fy2018, levels_fy2019, levels_fy2020, levels_fy2021]).reset_index(drop=True), on=['account_id', 'fiscal_year'])
donor_details['currentVlapsed'] = np.where(donor_details.retention_status.isin(['Acquired', 'Retained', 'Reactivated']), 'Current', donor_details.retention_status)
donor_details

In [ ]:
donor_details.retention_status.value_counts()

In [ ]:
donor_details.groupby(['fiscal_year'])['retention_status'].value_counts()

In [ ]:
donor_details['currentVlapsed'].value_counts()

In [ ]:
donor_details.groupby(['fiscal_year'])['currentVlapsed'].value_counts()

In [ ]:
#Calculate the current to lapsed ration by fy and add to donor_details df
donor_details.groupby(['fiscal_year'])['currentVlapsed'].value_counts(normalize=True)

In [ ]:
ax = sns.set(style="ticks", palette="pastel") #Update sns version to access set_theme attribute, but investigate all implications before updating te version

# Load the example tips dataset
#tips = sns.load_dataset("tips")

# Draw a nested boxplot to show bills by day and time
ax = sns.boxplot(x="fiscal_year", y="stp_total",
            hue="retention_status", palette='winter',
            data=donor_details)
#sns.despine(offset=10, trim=True)

In [ ]:
# >>> g = sns.catplot(x="sex", y="total_bill",
# ...                 hue="smoker", col="time",
# ...                 data=tips, kind="box",
# ...                 height=4, aspect=.7);

In [ ]:
# sns.set(style="ticks", palette="pastel") #Update sns version to access set_theme attribute, but investigate all implications before updating te version

# # Load the example tips dataset
# tips = sns.load_dataset("tips")

# # Draw a nested boxplot to show bills by day and time
# sns.boxplot(x="day", y="total_bill",
#             hue="smoker", palette=["m", "g"],
#             data=tips)
# sns.despine(offset=10, trim=True)

In [ ]:
ax = stp_level_dist.iloc[1:,5:].plot(kind='bar', stacked=True)

In [ ]:
donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns='retention_status', aggfunc='sum').plot(kind='bar', stacked='True')

#pd.pivot_table(values=donor_details., columns=donor_details['stp_level'], index=donor_details['fiscal_year'])
#ax = donor_details.loc[:, ['retention_status', 'stp_level']].plot(kind='bar')

In [ ]:
donor_details[donor_details['retention_status'] != 'Lapsed'].pivot_table(values = 'stp_total', index='fiscal_year', columns='retention_status', aggfunc='sum').T.plot(kind='barh', stacked='True')


In [ ]:
donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns='retention_status', aggfunc='sum').T.plot(kind='barh', stacked='True')

In [ ]:
donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns='retention_status', aggfunc='sum').T

In [ ]:
donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns=['retention_status', 'stp_level'], aggfunc='sum').T

In [ ]:
donor_details.describe(include='all').T

# Calculate retention rate by stp_level and fy

In [ ]:
sns.catplot(x='stp_level',col='retention_status',kind='count',data=donor_details)

In [ ]:
donor_details.loc[(donor_details.fiscal_year == '2017') & (donor_details.stp_level == 'Supporter'), 'account_id']

In [ ]:
g = donor_details[donor_details['fiscal_year'] == '2021'].groupby("stp_level").mean()
mean_y = donor_details.ret_rate_xfy.mean()
feature = 'ret_rate_xfy'
fig = plt.figure(figsize = (10,10))
plt.title('Bar plot of marital  vs mean ' + feature, fontsize= 20)
d = g[feature]
d.plot(kind = 'bar', edgecolor = 'black')
plt.hlines(mean_y, xmin = -1, xmax = donor_details[donor_details['fiscal_year'] == '2021'].stp_level.nunique(), label = 'avg target value')
plt.xlabel("stewardship level", fontsize= 18)
plt.ylabel('average retention rate', fontsize= 18)
plt.xticks(fontsize= 12, rotation = 45)
plt.yticks(fontsize= 12)
plt.legend()
plt.show()

In [ ]:
# g = df.groupby("marital").mean()
# mean_y = df.y.mean()
# feature = 'y'
# fig = plt.figure(figsize = (10,10))
# plt.title('Bar plot of marital  vs mean ' + feature, fontsize= 20)
# d = g[feature]
# d.plot(kind = 'bar', edgecolor = 'black')
# plt.hlines(mean_y, xmin = -1, xmax = df.job.nunique(), label = 'avg target value')
# plt.xlabel("marital categories", fontsize= 18)
# plt.ylabel('average target value', fontsize= 18)
# plt.xticks(fontsize= 12, rotation = 45)
# plt.yticks(fontsize= 12)
# plt.legend()
# plt.show()

In [ ]:
stp_stats_fy = donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns='stp_level', aggfunc='sum').T.set_index(pd.Index(['Did Not Give', 'Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'])).iloc[1:,]
type(stp_stats_fy)

In [ ]:
stp_stats_fy.index

In [ ]:
stp_stats_fy.values

In [ ]:
stp_stats_fy.columns

In [ ]:
donor_details.head()

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details,
            kind='box',
            hue='stp_level')

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details,
            kind='box',
            hue='stp_level',
           col='stp_level',
           col_wrap=2,
           sharex = True,
            sharey=False)

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            kind='box',
            hue='fiscal_year',
           col='stp_level',
           col_wrap=1,
            col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       'Director', 'Producer'],
           sharex = True,
            sharey=False,
           legend_out=True)

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            kind='box',
            hue='fiscal_year',
           col='stp_level',
           col_wrap=3,
            col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       'Director', 'Producer'],
           sharex = True,
            sharey=False,
           legend_out=True)

In [ ]:
#Summary stats showing concrete numbers of graphs above
donor_details.groupby(['stp_level', 'fiscal_year'])['stp_total'].describe().T

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            kind='swarm',
            hue='fiscal_year',
           col='stp_level',
           col_wrap=1,
            col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       'Director', 'Producer'],
           sharex = False,
            sharey=False,
           legend_out=True)
plt.show()

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[(donor_details.stp_level != 'Did Not Give') & (donor_details.stp_level != 'Producer')],
            kind='swarm',
            hue='stp_level', hue_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'],
           #col='stp_level',
           #col_wrap=1,
            #col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       #'Director', 'Producer'],
           sharex = True,
            sharey=True,
           legend_out=True)
plt.title('Distribution of Stewardship Giving Totals Level by Level and Fiscal Year')
plt.show()

In [ ]:
sns.catplot(x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[(donor_details.stp_level != 'Did Not Give')],
            kind='swarm',
            hue='stp_level', hue_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'],
           #col='stp_level',
           #col_wrap=1,
            #col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       #'Director', 'Producer'],
           sharex = True,
            sharey=True,
           legend_out=True)
plt.title('Distribution of Stewardship Giving Totals Level by Level and Fiscal Year')
plt.show()

In [ ]:
sns.catplot(ax=ax, x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            kind='point',
            hue='stp_level',
           #col='stp_level',
           #col_wrap=1,
            #col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       #'Director', 'Producer'],
           sharex = True,
            sharey=False,
           legend_out=True)
plt.show()

In [ ]:
#.set_index(pd.Index(['Did Not Give', 'Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'])).iloc[1:,]
donor_details.loc[donor_details.stp_level != 'Did Not Give'].set_index(pd.Index(['Did Not Give', 'Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'])).iloc[1:,]

In [ ]:
fig, ax = plt.subplots(figsize=(15,20))
ax = sns.pointplot(ax=ax, x='stp_level',
            y='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            #kind='point',
            hue='fiscal_year',
            order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'],
           alpha=.5,
                   #col='stp_level',
           #col_wrap=1,
            #col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       #'Director', 'Producer'],
           sharex = True,
            sharey=False,
           legend_out=True)
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(15,20))
ax = sns.pointplot(ax=ax, x='fiscal_year',
            y='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            #kind='point',
            hue='stp_level',
           #col='stp_level',
           #col_wrap=1,
            #col_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer',
       #'Director', 'Producer'],
           sharex = True,
            sharey=False,
           legend_out=True)
plt.show()

In [ ]:
g = sns.catplot(y='stp_level',
            x='stp_total',
            data=donor_details.loc[donor_details.stp_level != 'Did Not Give'],
            kind='box',
            hue='stp_level',
           col='fiscal_year',
           col_wrap=2,
           sharex = False,
            sharey=False,
           legend_out=True)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

**Reorder Y Axes Above**

In [ ]:
donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns='stp_level', aggfunc='sum').plot(kind='bar')

In [ ]:
donor_details.pivot_table(values = 'stp_total', index='fiscal_year', columns=['stp_level', 'retention_status'], aggfunc='sum').T.plot(kind='bar')

In [ ]:
#Check that all stewardship levels are present in each season
level_cols = ['stp_total_s28_level', 'stp_total_s27_level', 'stp_total_s26_level', 'stp_total_s25_level', 'stp_total_s24_level',
              'stp_toDate_s28_level', 'stp_toDate_s27_level', 'stp_toDate_s26_level', 'stp_toDate_s25_level', 'stp_toDate_s24_level']
for col in level_cols:
    print(levels_by_season_df[col].value_counts(dropna=False))

In [ ]:
#Check that all stewardship levels are present in each season (display as relative frequencies)
level_cols = ['stp_total_s28_level', 'stp_total_s27_level', 'stp_total_s26_level', 'stp_total_s25_level', 'stp_total_s24_level',
              'stp_toDate_s28_level', 'stp_toDate_s27_level', 'stp_toDate_s26_level', 'stp_toDate_s25_level', 'stp_toDate_s24_level']
for col in level_cols:
    print(levels_by_season_df[col].value_counts(normalize = True, dropna=False))

In [ ]:
stp_allDonors_s24 = levels_by_season_df.groupby('stp_total_s24_level')['account_id'].count().rename('stp_total_s24_level')

In [ ]:
stp_allDonors_s25 = levels_by_season_df.groupby('stp_total_s25_level')['account_id'].count().rename('stp_total_s25_level')

In [ ]:
#stp_allDonors_s25.loc['Producer'] = 0

In [ ]:
stp_allDonors_s25

In [ ]:
stp_allDonors_s26 = levels_by_season_df.groupby('stp_total_s26_level')['account_id'].count().rename('stp_total_s26_level')

In [ ]:
stp_allDonors_s27 = levels_by_season_df.groupby('stp_total_s27_level')['account_id'].count().rename('stp_total_s27_level')

In [ ]:
stp_allDonors_s28 = levels_by_season_df.groupby('stp_total_s28_level')['account_id'].count().rename('stp_total_s28_level')

In [ ]:
stp_toDateDonors_s24 = levels_by_season_df.groupby('stp_toDate_s24_level')['account_id'].count().rename('stp_toDate_s24_level')

In [ ]:
stp_toDateDonors_s25 = levels_by_season_df.groupby('stp_toDate_s25_level')['account_id'].count().rename('stp_toDate_s25_level')

In [ ]:
#stp_toDateDonors_s25.loc['Producer'] = 0

In [ ]:
stp_toDateDonors_s26 = levels_by_season_df.groupby('stp_toDate_s26_level')['account_id'].count().rename('stp_toDate_s26_level')

In [ ]:
#stp_toDateDonors_s26.loc['Producer'] = 0

In [ ]:
stp_toDateDonors_s27 = levels_by_season_df.groupby('stp_toDate_s27_level')['account_id'].count().rename('stp_toDate_s27_level')

In [ ]:
stp_toDateDonors_s28 = levels_by_season_df.groupby('stp_toDate_s28_level')['account_id'].count().rename('stp_toDate_s28_level')

In [ ]:
#Concatenate the series above to facilitate visualizing stewardship levels by season
stp_level_dist = (pd.concat([stp_allDonors_s24, stp_allDonors_s25, stp_allDonors_s26, stp_allDonors_s27, stp_allDonors_s28, stp_toDateDonors_s24, stp_toDateDonors_s25, stp_toDateDonors_s26, stp_toDateDonors_s27, stp_toDateDonors_s28], axis=1, join='inner', copy=False))
stp_level_dist

In [ ]:
#Change row index to arrange the levels in proper order.
stp_level_dist = stp_level_dist.loc[['Did Not Give', 'Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']]
stp_level_dist

**Merge aFund df with a subset of donor_details to append stewardship level and retention metrics to former in which each row is a donation.**

In [ ]:
aFund.shape, donor_details.shape

In [ ]:
keep=[0,3,5,7,8,10,11,13,16, 17, 18, 19, 20,22,23,26,28,29,30,31,32]
np.take(aFund.columns, keep)

In [ ]:
#Prepare aFund subset to join with donor_details df
#Select relevant columns and filter out rows where fiscal year is 2016 (2016 not included in donor_details df)
# aFund_subset = aFund.loc[,['account_id', 'account_record_type', 'donation_record_type', 'amount',
#        'close_date', 'mailing_city', 'mailing_state', 'fund', 'type', 'stage',
#        'fiscal_year', 'fiscal_quarter', 'last_donation_amount', 'close_month',
#        'anonymous', 'matching_amount', 'lifetime_donation_total_amount',
#        'lifetime_donation_count', 'lifetime_subscription_purchase_total',
#        'lifetime_single_ticket_purchase_total', 'lifetime_order_count']]
# aFund_subset.sort_values('fiscal_year').reset_index(drop=True)

In [ ]:
#Prepare aFund subset to join with donor_details df
#Select relevant columns and filter out rows where fiscal year is 2016 (2016 not included in donor_details df)
aFund_subset = aFund.loc[:,['account_id', 'account_record_type', 'donation_record_type', 'amount',
       'close_date', 'mailing_city', 'mailing_state', 'fund', 'type', 'stage',
       'fiscal_year', 'fiscal_quarter', 'last_donation_amount', 'close_month',
       'anonymous', 'matching_amount', 'lifetime_donation_total_amount',
       'lifetime_donation_count', 'lifetime_subscription_purchase_total',
       'lifetime_single_ticket_purchase_total', 'lifetime_order_count']]
aFund_subset.sort_values('fiscal_year').reset_index(drop=True)

# Add donor_detail columns to aFund_subset to faciliate deeper analysis.

In [ ]:
aFund_donations = pd.merge(aFund_subset, donor_details, how='left', on=['account_id', 'fiscal_year']).sort_values(by=['fiscal_year', 'account_id']).reset_index(drop=True)
aFund_donations.head()

In [ ]:
donor_details_afund = pd.merge(donor_details, aFund_subset, how='left', on=['account_id', 'fiscal_year']).sort_values(by=['fiscal_year', 'account_id']).reset_index(drop=True)
donor_details_afund.head()

In [ ]:
#Create dataframe of af donation totals by fiscal year and the contributors' stewardship level
totals_fyStp = donor_details_afund.groupby(['fiscal_year', 'stp_level'])['amount'].sum().reset_index()
totals_fyStp

# Use the 3 below (after perfecting them)

In [ ]:
totals_fyStp.pivot_table(values='amount', index='fiscal_year', columns='stp_level').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].columns

In [ ]:
#Create dataframe of af donation totals by fiscal year and the contributors' stewardship level
#totals_fyStp = donor_details_afund.groupby(['fiscal_year', 'stp_level'])['amount'].sum().reset_index()


#Plot the annual fund medians by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
g = sns.barplot(x=totals_fyStp.pivot_table(values='amount', index='fiscal_year', columns='stp_level').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].index, 
            y=totals_fyStp.pivot_table(values='amount', index='fiscal_year', columns='stp_level').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].values, 
            hue = totals_fyStp.pivot_table(values='amount', index='fiscal_year', columns='stp_level').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].columns,
           dodge=False)
#Add legend title (line below + setting the sns.barplot object equal to 'g' as above)
g.legend().set_title('Stewardship Level')

#Assign axis labels:
plt.ylabel('Median Donation ($)', fontsize=14)
plt.xlabel('Fiscal Quarter', fontsize=14)
plt.xticks(rotation=45)
plt.title('Annual Fund Donation Median By Fiscal Quarter', fontsize=15)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 1
# Annotating the bar plot with the values 
# for i in range(len(['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
#                      'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021'])):
#     plt.annotate(round(aFund.groupby(['fiscal_quarter'])['amount'].median().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
#                      'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']], 2).values[i], (-0.1 + i, aFund.groupby(['fiscal_quarter'])['amount'].median().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
#                      'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i] + j),
#                 horizontalalignment='center',
#                 fontsize=13)

In [ ]:
ax = totals_fyStp.pivot_table(values='amount', index='fiscal_year', columns='stp_level').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].plot(kind='bar', stacked=True)
plt.title("Each Stewardship Level Tends to Contribute More Money Over Time")
plt.xlabel("Fiscal Year")
plt.xticks(rotation=45)
plt.ylabel("Donation Totals ($) by Fiscal Year")
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)).set_title('Stewardship Level')

In [ ]:
#Plot each stewardship level's contribution to the annual fund totals by fiscal year
ax = pd.crosstab(index=totals_fyStp.fiscal_year, columns=totals_fyStp.stp_level, values=totals_fyStp.amount, aggfunc='max', normalize='index').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']].plot(kind='bar', stacked=True, rot=45)
plt.title("'Performers' Consistenty Account for the Largest % of Annual Fund Gift By Fiscal Year")
plt.xlabel("Fiscal Year")
plt.xticks(rotation=45)
plt.ylabel("Proportion of Total Donation")
ax.legend(loc='center left', bbox_to_anchor=(1.0, 0.5)).set_title('Stewardship Level')

In [ ]:
pd.crosstab(index=totals_fyStp.fiscal_year, columns=totals_fyStp.stp_level, values=totals_fyStp.amount, aggfunc='max', normalize='index').loc[:,['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer']]

In [ ]:
g = sns.catplot(x="fiscal_year", y="number_of_donations_byFY",
                 hue="retention_status", col="stp_level", col_wrap=2,
                 data=donor_details_afund[donor_details_afund.stp_level != 'Did Not Give'], kind="box",
                 height=10, aspect=1.5)

In [ ]:
g = sns.catplot(x="fiscal_year", y="number_of_donations_byFY",
                 hue="retention_status", col="stp_level", col_wrap=2,
                 data=donor_details_afund[donor_details_afund.stp_level != 'Did Not Give'], kind="point",
                 height=10, aspect=1.5)


In [ ]:
g = sns.catplot(x="fiscal_year", y="donor_total_byFY",
                 hue="stp_level", hue_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'],
                 data=donor_details_afund[donor_details_afund.stp_level != 'Did Not Give'], kind="point",
                 height=10, aspect=1.5)

In [ ]:
g = sns.catplot(x="fiscal_year", y="number_of_donations_byFY",
                 hue="stp_level", hue_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'],
                 data=donor_details_afund[donor_details_afund.stp_level != 'Did Not Give'], kind="point",
                 height=10, aspect=1.5)

In [ ]:

g = sns.catplot(x="fiscal_year", y="number_of_donations_byFY",
                 hue="retention_status", col="stp_level", col_wrap=2,
                 data=donor_details_afund[donor_details_afund.stp_level != 'Did Not Give'], kind="swarm",
                 height=10, aspect=1.5)

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(x=aFund.groupby(['fiscal_year'])['amount'].sum().index, y=aFund.groupby(['fiscal_year'])['amount'].sum().values, color='green')

#Assign axis labels:
plt.ylabel('Donation Totals ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 10
# Annotating the bar plot with the values (means by fy)
for i in range(len(np.sort(aFund.fiscal_year.unique()))):
    plt.annotate(round(aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']], 2).values[i],
                 (-0.1 + i, aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
#Plot the annual fund totals by fiscal quarter

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[20,8])

#Line Plot
g = sns.barplot(x=aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].index, 
            y=aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values, 
            hue = fq_wfy['fiscal_year'].values,
           dodge=False)
g.legend(loc='upper left').set_title('RTP Fiscal Year')
#Assign axis labels:
plt.ylabel('Donations Totals ($)', fontsize=15)
plt.xlabel('Fiscal Quarter', fontsize=15)
plt.xticks(rotation=45)
plt.title('Annual Fund Donation Totals By Fiscal Quarter', fontsize=17)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 2000
# Annotating the bar plot with the values 
for i in range(len(['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021'])):
    plt.annotate(round(aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']], 2).values[i], (-0.1 + i, aFund.groupby(['fiscal_quarter'])['amount'].sum().loc[['Q1-2016', 'Q2-2016', 'Q3-2016','Q4-2016', 'Q1-2017', 'Q2-2017', 'Q3-2017','Q4-2017','Q1-2018', 'Q2-2018', 'Q3-2018','Q4-2018', 'Q1-2019', 'Q2-2019', 'Q3-2019','Q4-2019',
                     'Q1-2020', 'Q2-2020', 'Q3-2020','Q4-2020', 'Q1-2021', 'Q2-2021', 'Q3-2021']].values[i] + j), 
                 horizontalalignment='center',
                 verticalalignment='top',
                 fontsize=13)

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(data=totals_fyStp, x='fiscal_year', y='amount', hue='stp_level', hue_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'], ci=None)

#Assign axis labels:
plt.ylabel('Donation Totals ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
i = 1.0
j = 10
# Annotating the bar plot with the values (means by fy)
for i in range(len(np.sort(aFund.fiscal_year.unique()))):
    plt.annotate(round(totals_fyStp['amount'].loc[['2017', '2018', '2019', '2020', '2021']], 2).values[i],
                 (-0.1 + i, totals_fyStp['amount'].loc[['2017', '2018', '2019', '2020', '2021']].values[i] + j), 
                 fontsize=13)
    
plt.show()

In [ ]:
totals_fyStp.plot(kind='bar', x='fiscal_year', y='amount', color='stp_level')

In [ ]:
# One liner to create a stacked bar chart.
ax = sns.histplot(totals_fyStp, x='fiscal_year', hue='stp_level', weights='amount',
             multiple='stack')
ax.set_ylabel('amount')
# Fix the legend so it's not on top of the bars.
legend = ax.get_legend()
legend.set_bbox_to_anchor((1, 1))

In [ ]:
#Plot the annual fund means by fiscal year

#set style
sns.set(style='darkgrid')

#Adjust the plot size
plt.figure(figsize=[15,6])

#Line Plot
sns.barplot(data=totals_fyStp, x='fiscal_year', y='amount', stacked=True, hue='stp_level', hue_order=['Pre-Steward', 'Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'], ci=None)

#Assign axis labels:
plt.ylabel('Donation Totals ($)', fontsize=14)
plt.xlabel('Fiscal Year', fontsize=14)
plt.title('Annual Fund Donation Totals By Fiscal Year (July 1 - June 30)', fontsize=14)

#Annotate
# This is the location for the annotated text
# i = 1.0
# j = 10
# # Annotating the bar plot with the values (means by fy)
# for i in range(len(np.sort(aFund.fiscal_year.unique()))):
#     plt.annotate(round(aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']], 2).values[i],
#                  (-0.1 + i, aFund.groupby(['fiscal_year'])['amount'].sum().loc[['2016', '2017', '2018', '2019', '2020', '2021']].values[i] + j), 
#                  fontsize=13)
    
plt.show()

In [ ]:
ax = sns.set(style="ticks") #Update sns version to access set_theme attribute, but investigate all implications before updating te version

# Load the example tips dataset
#tips = sns.load_dataset("tips")

# Draw a nested boxplot to show bills by day and time
ax = sns.boxplot(x="fiscal_year", y="stp_total",
            hue='currentVlapsed', palette=["m", "g"],
            data=donor_details)
#sns.despine(offset=10, trim=True)

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x="fiscal_year", y="stp_total",
            hue='currentVlapsed',
            data=donor_details)

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x='stp_level', y="stp_total",
            hue='currentVlapsed', col="fiscal_year",
            data=donor_details)
plt.xticks(rotation=45)

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x='stp_level', y="stp_total",
            hue='currentVlapsed', col="fiscal_year", col_wrap=2,
            data=donor_details[donor_details.stp_level != 'Did Not Give'],  height=3, aspect=1.5, jitter=True)
plt.xticks(rotation=45)

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x='stp_level', y="stp_total",
            hue='currentVlapsed', col="fiscal_year", col_wrap=2,
            data=donor_details[donor_details.stp_level != 'Did Not Give'],  height=3, aspect=1.5)
plt.xticks(rotation=45)

In [ ]:
ax = stp_level_dist.iloc[1:,5:].plot(kind='bar', stacked=True)

In [ ]:
ax = stp_level_dist.iloc[2:,0:5].T.plot(kind='bar', stacked=True)

In [ ]:
ax = stp_level_dist.iloc[2:,0:5].T.plot(kind='bar', stacked=False)

In [ ]:
ax = stp_level_dist.iloc[2:,0:5].T.set_index(pd.Index(['2017', '2018', '2019', '2020', '2021'])).plot(kind='bar', stacked=False) 

In [ ]:
ax = stp_level_dist.iloc[1:,0:5].T.set_index(pd.Index(['2017', '2018', '2019', '2020', '2021'])).plot(kind='bar', stacked=False)
ax.legend(loc='upper left')#.set_title('Stewardship Level')
plt.show()

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x='stp_level', y="stp_total",
            hue='retention_status', col="fiscal_year",
            data=donor_details)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:

sns.set(style="ticks")
g = sns.catplot(x='stp_level', y="donor_median_byFY",
            hue='retention_status', col="fiscal_year", kind='box',
            col_wrap=2,
            data=donor_details[donor_details.stp_level != 'Did Not Give'],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)


In [ ]:
sns.set(style="ticks")
g = sns.catplot(x="fiscal_year", y="donor_median_byFY",
            hue='currentVlapsed', col='stp_level' , kind='box',
            col_wrap=2,
            data=donor_details[donor_details.stp_level != 'Did Not Give'],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)


In [ ]:
sns.set(style="ticks")
g = sns.catplot(x="fiscal_year", y="donor_median_byFY",
            hue='currentVlapsed', col='stp_level' , kind='box',
            col_wrap=2,
            data=donor_details.loc[donor_details.stp_level.isin(['Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'])],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)


In [ ]:
sns.set(style="ticks")
g = sns.catplot(x="fiscal_year", y="donor_median_byFY",
            hue='currentVlapsed', col='stp_level' , kind='box',
            col_wrap=1,
            data=donor_details.loc[donor_details.stp_level.isin(['Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'])],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x="fiscal_year", y="number_of_donations_byFY",
            hue='currentVlapsed', col='stp_level' , kind='box',
            col_wrap=2,
            data=donor_details.loc[donor_details.stp_level.isin(['Supporter', 'Investor', 'Underwriter', 'Performer', 'Director', 'Producer'])],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:
sns.set(style="ticks")
g = sns.catplot(x="fiscal_year", y="donor_median_byFY",
            hue='retention_status', col='stp_level' , kind='box',
            col_wrap=2,
            data=donor_details[donor_details.stp_level != 'Did Not Give'],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)


In [ ]:
sns.set(style="ticks")
g = sns.catplot(x='stp_level', y="stp_total",
            hue='retention_status', col="fiscal_year", kind='box',
            col_wrap=2,
            data=donor_details[donor_details.stp_level != 'Did Not Give'],  height=3, aspect=1.5)
#plt.xticks(rotation=45)
for ax in g.axes.flat:
    for label in ax.get_xticklabels():
        label.set_rotation(45)

In [ ]:
stp_level_dist.iloc[2:,0:5].T.set_index(pd.Index(['2017', '2018', '2019', '2020', '2021']))

In [ ]:
stp_level_dist.iloc[2:,0:5].T.set_index(pd.Index(['2017', '2018', '2019', '2020', '2021'])).plot(kind='bar', stacked=False)

In [ ]:
ax = stp_level_dist.iloc[2:,0:5].plot(kind='bar', stacked=False)

In [ ]:
ax = stp_level_dist.iloc[2:,0:5].T.plot(kind='bar', stacked=False)

# Facet wrap w/ fiscal year

In [ ]:
ax = stp_level_dist.T.iloc[0:4,:].plot(kind='bar', stacked=True)

In [ ]:
ax = stp_level_dist.T.iloc[0:4,:].plot(kind='bar', stacked=False)

In [ ]:
stp_level_dist.T

In [ ]:
ax = stp_level_dist.T.iloc[:5,1:].set_index(pd.Index(['Season 24', 'Season 25', 'Season 26', 'Season 27', 'Season 28'])).plot(kind='bar', stacked=False, figsize=(20,12), title='Stewardship Level Volume by Season - Full Giving Periods')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

# ADD DOLLAR AMOUNTS TO STEWARDSHIP PERIOD LEGENDS & ANNOTATE GIVING PERIOD UNDER REVIEW

In [ ]:
ax = stp_level_dist.T.iloc[:5,2:].set_index(pd.Index(['Season 24', 'Season 25', 'Season 26', 'Season 27', 'Season 28'])).plot(kind='bar', stacked=False, figsize=(20,12), title='Stewardship Level Volume by Season - Full Giving Periods')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
ax = stp_level_dist.T.iloc[5:,1:].set_index(pd.Index(['Season 24', 'Season 25', 'Season 26', 'Season 27', 'Season 28'])).plot(kind='bar', stacked=False, figsize=(20,12), title='Stewardship Level Volume by Season - Full Giving Periods')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
ax = stp_level_dist.T.iloc[5:,2:].set_index(pd.Index(['Season 24', 'Season 25', 'Season 26', 'Season 27', 'Season 28'])).plot(kind='bar', stacked=False, figsize=(20,12), title='Stewardship Level Volume by Season - Common Date Range')
for p in ax.patches:
    ax.annotate(str(p.get_height()), (p.get_x() * 1.005, p.get_height() * 1.005))

In [ ]:
ax = stp_level_dist.T.iloc[5:,2:].plot(kind='bar', stacked=False, subplots=True, layout=(4,2), legend=False, title='Stewardship Level Volume by Season - Common Date Range')

In [ ]:
ax = stp_level_dist.T.iloc[5:,2:].plot(kind='box')

In [ ]:
ax = stp_level_dist.T.iloc[5:,2:].set_index(pd.Index(['Season 24', 'Season 25', 'Season 26', 'Season 27', 'Season 28'])).T.plot(kind='box')


In [ ]:
#Let's look specifically at the new stewardship total columns
aFund[['account_name','stp_total_s28', 'stp_total_s27', 'stp_total_s26', 'stp_total_s25']].sort_values(by='stp_total_s28')

In [ ]:
figure, ax = plt.subplots(1,1, figsize=(20,24), squeeze=False)

#See the distrubution of the data

sns.distplot(donations['amount'],ax= ax[0,0])
plt.show()

In [ ]:
plt.hist(donations.amount[donations.amount.notnull()], color = 'blue', edgecolor = 'black', bins = int(25000/250))

In [ ]:
sns.distplot(donations.amount[donations.amount.notnull()], hist=True, kde=False, 
             bins=int(25000/250), color = 'blue',
             hist_kws={'edgecolor':'black'})
# Add labels
#plt.title('Histogram of Arrival Delays')
#plt.xlabel('Delay (min)')
#plt.ylabel('Flights')

In [ ]:
donations.amount[donations.amount.notnull()]

In [ ]:
donations.isna().sum()